# Initial Imports

In [1]:
import argparse, sys, os, logging
import numpy as np
import pandas as pd
import powergrid_data
from hmmlearn import hmm
from datetime import datetime
from scipy import stats, special
from pandas.tools.plotting import autocorrelation_plot
from sklearn.preprocessing import StandardScaler, normalize
from statsmodels.nonparametric.smoothers_lowess import lowess
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, WeekdayLocator, DayLocator, HourLocator, AutoDateLocator, DateFormatter, AutoDateFormatter, date2num
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from analysis import analyzer
import warnings


%matplotlib inline
warnings.filterwarnings('ignore')

# Test and Train Files Directory

In [2]:
train = 'data/train/train.csv'
test = 'data/test/test_v1.csv'

# Train Model

In [3]:
power_grid = powergrid_data.datasets(train, test)

In [4]:
def pointAnomalyAnalyzer(power_grid, fullYear=None, fullMonth=[None], fullDay=None, threshold=1, n_comp=3, enable_meanStates = False):

    analyze = analyzer(power_grid)

    dates_columns = 'DateTime'
    features_columns = ['Global_active_power']

    analyze.time_series(dates_columns, features_columns)


    ## If you modify year, month and day, make sure the above dates_columns is either Date or DateTime

    ## Both test and train has data of year 2006, 2007 and 2008
    Year = fullYear

    ## Months takes a string of month full name e.g 'March'
    Month = fullMonth
    Day = fullDay

    ## If you modify hour, minutes and seconds, make sure the above dates_columns is either DateTime or Time
    ## Hour uses 24 hr clock
    Hour = None
    Minutes = None
    Seconds = None

    train_dates, train_features, test_dates, test_features = analyze.parser(year = Year, month = Month, day = Day, hour = Hour, minutes = Minutes, seconds = Seconds)


    # Make an HMM instance and execute fit

    init_probability = np.array([0.6, 0.3, 0.1])

    translation_probability = np.array([
        [0.7, 0.2, 0.1],
        [0.3, 0.5, 0.2],
        [0.3, 0.3, 0.4]]
    )

    model = hmm.GaussianHMM(n_components=n_comp, covariance_type="full", tol = 0.1, n_iter=500, init_params="st")
    model.startprob_ = init_probability
    model.transmat_ = translation_probability

    model.fit(train_features)

    train_state_seq = model.sample(test_features.size)[1]
    test_state_seq = model.predict(test_features)

    means = []
    variance = []

    for i in range(model.n_components):
        means.append(model.means_[i].flatten()[0])
        variance.append(model.covars_[i].flatten()[0])

    standard_deviation = np.sqrt(variance)

    anom = []
    notanom = []
    meanStates = []

    test_observation = test_features.values

    

    for i in range(test_observation.size):
        state = test_state_seq[i]
        if (np.abs(means[state] - test_observation[i]) <= threshold):
            notanom.append(True)
        else:
            notanom.append(False)
        meanStates.append(means[state])

    test_features_anomaly = np.copy(test_observation)
    test_features_anomaly[notanom] = None
#     0 is normal, 1 is anomally
    AnomolyBinary = np.array(notanom) == False
    
    return AnomolyBinary.astype(int)




In [5]:
def outputFile(filename, threshold):
    result = []
    for year in power_grid._test_data['DateTime'].dt.year.unique():
        result.extend(pointAnomalyAnalyzer(power_grid=power_grid, fullYear=year, threshold=threshold))
    output = pd.DataFrame({'Anomaly Status' : result, 'Probability' : result})
    output.to_csv(filename, index=False, header=True)


In [6]:
# thresholds 0.5, 1, 1.25 ,1.5, 1.75 and 2

outputFile('T0.5_'+'PointAnomaly.csv', 0.5)
outputFile('T1_'+'PointAnomaly.csv', 1)
outputFile('T1.25_'+'PointAnomaly.csv', 1.25)
outputFile('T1.5_'+'PointAnomaly.csv', 1.5)
outputFile('T1.75_'+'PointAnomaly.csv', 1.75)
outputFile('T2_'+'PointAnomaly.csv', 2)



# Windowing Output

In [7]:
from itertools import islice
# Ref: https://stackoverflow.com/questions/6822725/rolling-or-sliding-window-iterator-in-python

def window(seq, n=2):
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield np.array(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield np.array(result)

def pointAnomalyAnalyzerWindowing(power_grid, fullYear=None, fullMonth=[None], fullDay=None, threshold=1, n_comp=3, windowParam = 5, enable_meanStates = False):

    analyze = analyzer(power_grid)

    dates_columns = 'DateTime'
    features_columns = ['Global_active_power']

    analyze.time_series(dates_columns, features_columns)


    ## If you modify year, month and day, make sure the above dates_columns is either Date or DateTime

    ## Both test and train has data of year 2006, 2007 and 2008
    Year = fullYear

    ## Months takes a string of month full name e.g 'March'
    Month = fullMonth
    Day = fullDay

    ## If you modify hour, minutes and seconds, make sure the above dates_columns is either DateTime or Time
    ## Hour uses 24 hr clock
    Hour = None
    Minutes = None
    Seconds = None

    train_dates, train_features, test_dates, test_features = analyze.parser(year = Year, month = Month, day = Day, hour = Hour, minutes = Minutes, seconds = Seconds)


    # Make an HMM instance and execute fit

    init_probability = np.array([0.6, 0.3, 0.1])

    translation_probability = np.array([
        [0.7, 0.2, 0.1],
        [0.3, 0.5, 0.2],
        [0.3, 0.3, 0.4]]
    )

    model = hmm.GaussianHMM(n_components=n_comp, covariance_type="full", tol = 0.1, n_iter=500, init_params="st")
    model.startprob_ = init_probability
    model.transmat_ = translation_probability

    #  window 5, 10, 15, and 20
    for w in window(train_features.values.flatten(), windowParam):
        train_window_features = w[:, np.newaxis]
        model.fit(train_window_features)

    train_state_seq = model.sample(test_features.size)[1]
    test_state_seq = model.predict(test_features)

    means = []
    variance = []

    for i in range(model.n_components):
        means.append(model.means_[i].flatten()[0])
        variance.append(model.covars_[i].flatten()[0])

    standard_deviation = np.sqrt(variance)

    anom = []
    notanom = []
    meanStates = []

    test_observation = test_features.values

    

    for i in range(test_observation.size):
        state = test_state_seq[i]
        if (np.abs(means[state] - test_observation[i]) <= threshold):
            notanom.append(True)
        else:
            notanom.append(False)
        meanStates.append(means[state])

    test_features_anomaly = np.copy(test_observation)
    test_features_anomaly[notanom] = None
#     0 is normal, 1 is anomally
    AnomolyBinary = np.array(notanom) == False
    
    return AnomolyBinary.astype(int)

In [8]:
# windowing 5, 10, 15, 20

def outputFileForWindowing(filename, wparam):
    result = []
    for year in power_grid._test_data['DateTime'].dt.year.unique():
        result.extend(pointAnomalyAnalyzerWindowing(power_grid=power_grid, fullYear=year, windowParam=wparam))
    output = pd.DataFrame({'Anomaly Status' : result, 'Probability' : result})
    output.to_csv(filename, index=False, header=True)


In [ ]:
outputFile('W5_'+'CollectiveAnomalyWindowing.csv', 5)
outputFile('W10_'+'CollectiveAnomalyWindowing.csv', 10)
outputFile('W15_'+'CollectiveAnomalyWindowing.csv', 15)
outputFile('W20_'+'CollectiveAnomalyWindowing.csv', 20)
